In [1]:
%pip install openai


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
from openai import OpenAI
import os
GPT_API_KEY = 'OPENAI_API_KEY'
os.environ['OPENAI_API_KEY'] = GPT_API_KEY
openai.api_key = GPT_API_KEY
client = OpenAI()


In [3]:
import pandas as pd
data=pd.read_csv("/Users/aryansahu/Desktop/IIT-P/DATASETS/Dailyhunt_Bangla.csv")


In [4]:
%pip install tiktoken
import tiktoken
tokenizer = tiktoken.get_encoding('cl100k_base')



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
def generate_headline(text, comments):
    # Ensure the text is a string
    if not isinstance(text, str):
        return None
    
    # Encode the text to get tokens
    tokens = tokenizer.encode(text)
    token_count = len(tokens)
    
    # Token limit
    token_limit = 1000
    
    # Trim the text if the token count exceeds the limit
    if token_count > token_limit:
        # Calculate the number of characters to retain
        text = text[:12000]
        # Recalculate tokens to ensure it's under the limit
        tokens = tokenizer.encode(text)
        while len(tokens) > token_limit:
            text = text[:-1]
            tokens = tokenizer.encode(text)
    
    # Call the model with the (possibly trimmed) text
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role": "system",
                "content": "Your task is to generate a headline which summarizes the text and the comments."
            },
            {
                "role": "user",
                "content": f"Generate a headline using the contents and comments, such that the headline includes the sentiment of comments as well. Generate it in the same language as the text. News-Headline.\n\nContents: {text}\n\nComments: {comments}\n\n."
            }
        ]
    )
    
    # Retrieve the generated headline
    headline = completion.choices[0].message.content
    return headline


In [7]:
# data['GPT headline'] = data['news_content'].apply(generate_headline)
data['GPT headline'] = data['Content'].apply(generate_headline)
data.to_csv('Bangla_Headlines.csv', index=False)
# print("DataFrame saved to 'headlines_with_content.csv'")

In [5]:
def generate_headline(text, comments):
    # Ensure the text is a string
    if not isinstance(text, str):
        return None
    
    # Encode the text to get tokens
    tokens = tokenizer.encode(text)
    token_count = len(tokens)
    
    # Token limit
    token_limit = 12000
    
    # Trim the text if the token count exceeds the limit
    if token_count > token_limit:
        # Calculate the number of characters to retain
        text = text[:12000]
        # Recalculate tokens to ensure it's under the limit
        tokens = tokenizer.encode(text)
        while len(tokens) > token_limit:
            text = text[:-1]
            tokens = tokenizer.encode(text)
    
    # Call the model with the (possibly trimmed) text
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role": "system",
                "content": "Your task is to generate a headline which summarizes the text."
            },
            {
                "role": "user",
                "content": f"Generate a headline using the contents and comments, such that the headline includes the sentiment of comments as well. Generate it in the same language as the text. News Headline.\n\nContents: {text}\n\nComments: {comments}\n\n."
            }
        ]
    )
    
    # Retrieve the generated headline
    headline = completion.choices[0].message.content
    return headline


In [6]:
# data['GPT headline'] = data['news_content'].apply(generate_headline)
data['GPT headline'] = data.apply(lambda row: generate_headline(row['Content'], row['Comments']), axis=1)
data.to_csv('Bangla_headlines_withcomments.csv', index=False)
print("DataFrame saved to 'headlines_with_content.csv'")

DataFrame saved to 'headlines_with_content.csv'
